# CASE INSTRUCTIONS

We are selling products online through resellers.
 
Most products are produced on demand, i.e. after an end customer has made the order. In order to reduce delivery times, we would like to keep certain products on stock so we can send them out immediately when we receive the order.
 
2 of our resellers (“Customer A”,”Customer B”) have kindly provided us with their latest sales data. Based on this data, could you recommend us which products we should keep on stock?
 
The task:<br>
<ul>
    <li>Please write a script that proposes the top 3 products to put on stock and explain your reasoning
    <li>Do you see alternative solutions?
    <li>What are the main challenges you see for this business case and how would you mitigate them?

________________________________________________________________________________________

# WHICH PRODUCTS TO PUT ON STOCK?

According to the information at my disposal, the best way to choose the top 3 products to put on stock is to assess the global quantity sold for each product as well as the number of sales for each product on the given period. 
The 3 references to put on stock will be references that <b>sell in high volumes and also have consistent demand across multiple transactions</b>. 

In order to do so, here's my reasoning: 

In [1]:
import pandas as pd
import numpy as np 

### PREPARATION REGARDING CUSTOMER A

In [2]:
# Read the Excel file and select the columns that will be useful for my work.

custA = pd.read_excel(r"C:\Users\kieny\Desktop\E+H\Customer A.xlsx", usecols = [0,1,2])
custA

,Invoice Date,E+H Item Number,QTY
0,2024-03-29 00:00:00,FMR50-AAACADBMRGF,1
1,2024-03-29 00:00:00,CPS11E-AA7BAA6,4
2,2024-03-29 00:00:00,CYK10-A031,8
3,2024-03-29 00:00:00,CM14-AAM,6
4,2024-03-28 00:00:00,FTL31-CA4M2AAVAJ,1
...,...,...,...
127,2024-03-01 00:00:00,PMP21-AA1V1SBWUJ,2
128,2024-03-01 00:00:00,PMP71B-CDBAEJH6BR3SJAAAQJH5+Q1Z1,12
129,2024-03-01 00:00:00,PMD75B-CDBAEJH37HJASAJA1B+NTNZ,3
130,2024-03-01 00:00:00,PMD75B-CDBAEJH37FJASAJA1B+NTNZ,9


In [3]:
# Delete last row of the dataframe as it will not be useful for the task.

custA.drop(custA.tail(1).index, inplace=True)

In [4]:
# Check for missing values in the dataframe.

print(custA.isna().sum())

Invoice Date       0
E+H Item Number    0
QTY                0
dtype: int64


In [5]:
# Check data types

custA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Invoice Date     131 non-null    object
 1   E+H Item Number  131 non-null    object
 2   QTY              131 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 3.2+ KB


In [6]:
# Modify the data type of columns when required. The 'Invoice Date' column was in an object type when it should be in a datetime type. 

custA['Invoice Date'] = pd.to_datetime(custA['Invoice Date']).dt.strftime('%Y%m%d')
custA['Invoice Date'] = pd.to_datetime(custA['Invoice Date'], format='%Y%m%d')
custA.head()

,Invoice Date,E+H Item Number,QTY
0,2024-03-29,FMR50-AAACADBMRGF,1
1,2024-03-29,CPS11E-AA7BAA6,4
2,2024-03-29,CYK10-A031,8
3,2024-03-29,CM14-AAM,6
4,2024-03-28,FTL31-CA4M2AAVAJ,1


In [7]:
# Check whether data types have been corrected as they should. We can see that 'Invoice Date' data type has been changed correctly.

custA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Invoice Date     131 non-null    datetime64[ns]
 1   E+H Item Number  131 non-null    object        
 2   QTY              131 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 3.2+ KB


### PREPARATION REGARDING CUSTOMER B

In [8]:
# Read the CSV file and select the columns that will be useful for my work.

custB = pd.read_csv(r"C:\Users\kieny\Desktop\E+H\Customer B.csv", sep = ";", header = 1, usecols=[4,5,8])
custB

,E+H Item Number,QTY,DATE
0,FMX11-CA11FS10,10.0,20240301.0
1,FTL31-AA4M2AAWBJ,11.0,20240304.0
2,FTW325-A2A1A,9.0,20240304.0
3,RLN22-8M2A,12.0,20240305.0
4,FTL31-AA4M2AAVBJ,4.0,20240306.0
5,FTW31-A1A5CA0A,3.0,20240307.0
6,RMA42-AAC,7.0,20240308.0
7,FTL31-AA4U3BAWSJ,2.0,20240313.0
8,RLN22-8M2A,4.0,20240313.0
9,FTW31-A1A5CA0A,3.0,20240319.0


In [9]:
# Delete the last row, as it contains only NaN values.

custB.drop(custB.tail(1).index, inplace=True)

In [10]:
# Verify null values and data types to adjust if necessary. 

custB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   E+H Item Number  20 non-null     object 
 1   QTY              20 non-null     float64
 2   DATE             20 non-null     float64
dtypes: float64(2), object(1)
memory usage: 608.0+ bytes


In [11]:
# Check for remaining missing values in the dataframe.

print(custB.isna().sum())

E+H Item Number    0
QTY                0
DATE               0
dtype: int64


In [12]:
# Several columns are not in the proper data type, so I correct them for future manipulation. 

custB['QTY'] = custB['QTY'].astype(int)
custB['DATE'] = pd.to_datetime(custB['DATE'], format='%Y%m%d')

In [13]:
# Verify that all data are now in a proper data type. 

custB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   E+H Item Number  20 non-null     object        
 1   QTY              20 non-null     int32         
 2   DATE             20 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int32(1), object(1)
memory usage: 528.0+ bytes


### CONCATENATION OF BOTH DATAFRAMES

In [14]:
# Use the same columns names in both dataframes to prepare for concatenation. 

custA = custA.rename(columns={'Invoice Date':'DATE'})

In [15]:
# Concatenate both dataframes to analyze total sales

custAB = pd.concat([custA, custB])
custAB

,DATE,E+H Item Number,QTY
0,2024-03-29,FMR50-AAACADBMRGF,1
1,2024-03-29,CPS11E-AA7BAA6,4
2,2024-03-29,CYK10-A031,8
3,2024-03-29,CM14-AAM,6
4,2024-03-28,FTL31-CA4M2AAVAJ,1
...,...,...,...
15,2024-03-29,FTL33-CA4M2ABVBJ,2
16,2024-03-30,FTL31-AA1S3AAVBJ,15
17,2024-03-30,DMA20-AAACA1,8
18,2024-03-30,DMA15-AAACA1,9


In [16]:
# Group sales by Item Number in order to see which products are the most sold and the number of transactions on the period. 
grouped_sales = custAB.groupby('E+H Item Number').agg({'QTY': 'sum', 'DATE': 'count'})

# Rename the columns properly for better understanding. 
grouped_sales.columns = ['Total Qty', 'Nb Transactions']

# Sort the dataframe according to total quantity sold in a decreasing order. 
grouped_sales_sort = grouped_sales.sort_values(by = ['Total Qty'], ascending = False)

# Display the first 15 rows.
grouped_sales_sort.head(15)

,Total Qty,Nb Transactions
E+H Item Number,,
FTL31-AA1S3AAVBJ,29,2
PMD55-FC21BD67FGJHAJA1A,25,4
DMA50-AAACA1,25,3
PMD55-FC21BD27CGJHAJA1A,24,6
DMA15-AAAAA1,22,2
DMA15-AAACA1,20,3
FTL51B-CDA4ABIAA1AJA1VCJ,19,2
71355708,18,1
DMA20-AAACA1,16,2


### RESULTS

As a result, we found that the top 3 products sold when combining the sales from resellers A and B are the following references:<ul> 
    <li> FTL31-AA1S3AAVBJ (29 products sold)
    <li> PMD55-FC21BD67FGJHAJA1A (25 products sold)	
    <li> DMA50-AAACA1 (25 products sold)</ul>	
    
However, the reference <i>PMD55-FC21BD27CGJHAJA1A</i> was sold in 6 different transactions on the period for a total volume of 24 products sold. We have a stronger interest in putting on stocks for this reference rather than <i>FTL31-AA1S3AAVBJ</i>, which was sold on 2 transactions for a total volume of 29 products, as recurring purchases are high as well as volume sold.   	

With the information at my disposal, <b>I would recommend to put on stock on these products as they sell well and on multiple transactions on the given period</b>:
<ul> 
    <li> PMD55-FC21BD67FGJHAJA1A	
    <li> DMA50-AAACA1
    <li> PMD55-FC21BD27CGJHAJA1A 
</ul>	

I have noticed during my work that some values in the 'E+H Item Number' columns have a different format than others, with only numeric characters. I assume these values are incorrect but have only a limited impact on the results provided.

# ALTERNATIVE SOLUTIONS

With the information at disposal and in the event that a distributor is able to manage its own stock, an alternative solution would rely on keeping the resellers information separated. Indeed, one reseller may have a particular customer base when compared to another. By combining the sales results of all resellers, we lose the uniqueness of each distributor's needs and we don't take customers "weight" into account. For example, customer A sold 794 products on the period when customer B sold only 137 products.

In our case, I decided to combine sales information as the sample was quite restricted with only 20+ sales for customer B. 

In [17]:
sumA = custA['QTY'].sum()
sumB = custB['QTY'].sum()
print('Quantity sold for customer A: ',sumA)
print('Quantity sold for customer B: ',sumB)

Quantity sold for customer A:  794
Quantity sold for customer B:  137


# CHALLENGES AND RECOMMENDATIONS

Here are the main challenges I identify for this business case:<ul> 
    <li><b>Production time</b>: We lack information about the production time required by every product. By including this information in the analysis, we could identify products with longer production time that are also selling well and regularly. These products would require particular attention when it comes to stock. </li><br>
    <li><b>Data span</b>: The two files only concern sales for the month of March 2024. With a larger data span, we could be able to identify patterns in the demand, or seasonality and adapt our stock accordingly. I would recommend carrying out an in-depth analysis over a wider period (a year or more).</li><br>
    <li><b>Products profitability</b>: Information about profit margins on each product are probably available at E+H so I recommend considering the profitability of each product by factoring in the profit margins along with sales volume. Higher-margin products might be more suitable for stocking, even if they have lower sales volumes.</li><br>
    <li><b>Data quality</b>: As mentionned previously, some information in the sales tables shared by our customers seems to be incorrect such as some products' item numbers. This may alter the quality of the results of our analysis. To minimise this risk, I recommend contacting the customer who provided us with this information to check with them which products are concerned when the product number seems wrong.</li><br>
    <li><b>Customer segments</b>: We could ask feedback to our resellers on products that are popular among their customers and have higher customer satisfaction ratings. We would need to segment customers based on their preferences, business typology, purchasing behavior and tailor stocking decisions to meet the needs of each customer segment.</li><br>
    <li><b>Inventory costs</b>: We don't have information about inventory costs in our case but keeping products in stock incurs inventory holding costs (storage, handling, obsolescence). I would recommend to conduct a cost-benefit analysis to evaluate the trade-offs between inventory costs and potential benefits of stocking each product.</li>
</ul>

<b>Products shelf life</b>: In our case, I assume the products don't have a shelf life, so there is no such impact on the stocking strategy as it would with consumer goods for example.

